# Initialization

In [2]:
import numpy as np
import random
import numpy as np
import copy

def print_pretty_matrix(title, matrix):
    if not isinstance(matrix, np.ndarray):
        raise TypeError("Input must be a NumPy array")
    
    rows, cols = matrix.shape
    print(f"\n\n{title}\n********")
    for row in range(rows):
        formatted_row = " | ".join(f"{matrix[row, col]:>10}" for col in range(cols))
        print(f"| {formatted_row} |")

In [20]:
# Parámetros del Grid World
grid_size = 4
start = (0, 0)
goal = (3, 3)
learning_rate = 0.0005
discount_factor = 0.9
epsilon = 0.1
episodes = 10000
target_update_frequency = 100
# Definir las acciones
actions = ["up", "down", "left", "right"]
action_to_index = {action: i for i, action in enumerate(actions)}

# Definir la matriz de recompensas del Grid World
rewards = np.full((grid_size, grid_size), -0.1)  # Penalización por movimiento
rewards[1, 3] = 1  # Recompensa positiva
rewards[2, 2] = 1  # Recompensa positiva
rewards[1, 1] = -1  # Recompensa negativa
rewards[3, 0] = -1  # Recompensa negativa
rewards[goal] = 100  # Recompensa por llegar a la meta

print_pretty_matrix("rewards", rewards)



rewards
********
|       -0.1 |       -0.1 |       -0.1 |       -0.1 |
|       -0.1 |       -1.0 |       -0.1 |        1.0 |
|       -0.1 |       -0.1 |        1.0 |       -0.1 |
|       -1.0 |       -0.1 |       -0.1 |      100.0 |


# Torch

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Definir la red neuronal para aproximar la función Q
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(grid_size * grid_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 4)  # 4 acciones posibles: arriba, abajo, izquierda, derecha

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# Inicializar la red DQN principal y la red objetivo
dqn = DQN()
target_dqn = copy.deepcopy(dqn)
optimizer = optim.Adam(dqn.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# Función para convertir el estado (posición en la cuadrícula) en un tensor de entrada para la red
def state_to_tensor(state):
    tensor = torch.zeros(grid_size * grid_size)
    tensor[state[0] * grid_size + state[1]] = 1.0
    return tensor.unsqueeze(0).float()

# Función para elegir la acción (con epsilon-greedy)
def choose_action(state):
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, 3)  # Acción aleatoria
    else:
        with torch.no_grad():
            q_values = dqn(state_to_tensor(state))
        return torch.argmax(q_values).item()

# Función para obtener la próxima posición dada una acción
def take_action(state, action):
    if action == 0:  # up
        return (max(state[0] - 1, 0), state[1])
    elif action == 1:  # down
        return (min(state[0] + 1, grid_size - 1), state[1])
    elif action == 2:  # left
        return (state[0], max(state[1] - 1, 0))
    elif action == 3:  # right
        return (state[0], min(state[1] + 1, grid_size - 1))

# Entrenamiento con Double DQN
for episode in range(episodes):
    state = start

    while state != goal:
        action = choose_action(state)
        next_state = take_action(state, action)

        # Obtener la recompensa correspondiente
        reward = rewards[next_state]

        # Calcular el valor Q objetivo usando la red objetivo
        with torch.no_grad():
            next_q_values = dqn(state_to_tensor(next_state))
            best_action = torch.argmax(next_q_values).item()
            target_q_value = reward + discount_factor * target_dqn(state_to_tensor(next_state))[0, best_action]

        # Obtener el valor Q estimado actual
        q_values = dqn(state_to_tensor(state))
        current_q_value = q_values[0, action]

        # Calcular la pérdida
        loss = loss_fn(current_q_value, target_q_value)

        # Actualizar la red principal
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Mover al siguiente estado
        state = next_state

    # Actualizar la red objetivo cada 'target_update_frequency' episodios
    if episode % target_update_frequency == 0:
        target_dqn.load_state_dict(dqn.state_dict())

    # Imprimir la pérdida cada 1000 episodios
    if episode % 1000 == 0:
        print(f"Episodio: {episode}, Pérdida: {loss.item()}")

# Mostrar la matriz de recompensas para referencia
print("\nMatriz de Recompensas:")
print(rewards)

Episodio: 0, Pérdida: 10028.9306640625
Episodio: 1000, Pérdida: 0.8057648539543152
Episodio: 2000, Pérdida: 0.6272516250610352
Episodio: 3000, Pérdida: 3.512993335723877
Episodio: 4000, Pérdida: 0.3909302353858948
Episodio: 5000, Pérdida: 0.27299731969833374
Episodio: 6000, Pérdida: 0.3558445870876312
Episodio: 7000, Pérdida: 0.2181273102760315
Episodio: 8000, Pérdida: 0.3518146276473999
Episodio: 9000, Pérdida: 0.07332314550876617

Matriz de Recompensas:
[[ -0.1  -0.1  -0.1  -0.1]
 [ -0.1  -1.   -0.1   1. ]
 [ -0.1  -0.1   1.   -0.1]
 [ -1.   -0.1  -0.1 100. ]]


In [6]:
loss.item()

6.058767318725586

In [7]:
for i in range(4):
    for j in range(4):
        print(f"Q({i},{j}): {dqn(state_to_tensor((i, j)))}")

Q(0,0): [[-0.07794925 -0.01008882  0.04600317  0.07517506]]
Q(0,1): [[-0.11044546 -0.00890867  0.063161    0.05893465]]
Q(0,2): [[-0.07694578 -0.07176758  0.06956495 -0.04668758]]
Q(0,3): [[-0.12292645  0.0629231   0.07387641 -0.01150181]]
Q(1,0): [[-0.03791503  0.183098    0.14410655 -0.00131273]]
Q(1,1): [[-0.11482402 -0.07017019  0.11718072  0.08417861]]
Q(1,2): [[-0.02801664  0.05317232  0.07353303  0.05540504]]
Q(1,3): [[-0.05168027 -0.03166492  0.03067819  0.08033122]]
Q(2,0): [[-0.05656049  0.01939723  0.14320491  0.04027523]]
Q(2,1): [[-0.14294015  0.23239294  0.09981627  0.11295213]]
Q(2,2): [[-0.09099971  0.12558104  0.03222947  0.01954882]]
Q(2,3): [[-0.14825185  0.03904763  0.05670653  0.07405663]]
Q(3,0): [[-0.00987893  0.09967978  0.0408778   0.08319162]]
Q(3,1): [[-0.10838757  0.10081992  0.00804136 -0.00882077]]
Q(3,2): [[-0.10095967 -0.12758033  0.04881566  0.06136369]]
Q(3,3): [[-0.18945417  0.08904378  0.0470854  -0.06094315]]


# Tensorflow

In [21]:
import tensorflow as tf

In [25]:
episodes = 100
target_update_frequency = episodes // 5

In [23]:
# Definir la red neuronal para aproximar la función Q
class DQN(tf.keras.Model):
    def __init__(self):
        super(DQN, self).__init__()
        self.fc1 = tf.keras.layers.Dense(128, activation='relu', input_shape=(grid_size * grid_size,))
        self.fc2 = tf.keras.layers.Dense(128, activation='relu')
        self.fc3 = tf.keras.layers.Dense(4)  # 4 acciones posibles: arriba, abajo, izquierda, derecha

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)

# Inicializar la red DQN principal y la red objetivo
dqn = DQN()
target_dqn = DQN()
target_dqn.set_weights(dqn.get_weights())  # Inicializar la red objetivo con los pesos de la red principal
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.MeanSquaredError()

# Función para convertir el estado (posición en la cuadrícula) en un tensor de entrada para la red
def state_to_tensor(state):
    tensor = np.zeros(grid_size * grid_size)
    tensor[state[0] * grid_size + state[1]] = 1.0
    return np.expand_dims(tensor, axis=0).astype(np.float32)

# Función para elegir la acción (con epsilon-greedy)
def choose_action(state):
    if random.uniform(0, 1) < epsilon:
        return random.randint(0, 3)  # Acción aleatoria
    else:
        q_values = dqn(state_to_tensor(state))
        return np.argmax(q_values.numpy())

# Función para obtener la próxima posición dada una acción
def take_action(state, action):
    if action == 0:  # up
        return (max(state[0] - 1, 0), state[1])
    elif action == 1:  # down
        return (min(state[0] + 1, grid_size - 1), state[1])
    elif action == 2:  # left
        return (state[0], max(state[1] - 1, 0))
    elif action == 3:  # right
        return (state[0], min(state[1] + 1, grid_size - 1))

# Entrenamiento con Double DQN
for episode in range(episodes):
    state = start

    while state != goal:
        action = choose_action(state)
        next_state = take_action(state, action)

        # Obtener la recompensa correspondiente
        reward = rewards[next_state]

        # Calcular el valor Q objetivo usando la red objetivo
        next_q_values = dqn(state_to_tensor(next_state))
        best_action = np.argmax(next_q_values.numpy())
        target_q_value = reward + discount_factor * target_dqn(state_to_tensor(next_state))[0, best_action].numpy()

        # Obtener el valor Q estimado actual
        with tf.GradientTape() as tape:
            q_values = dqn(state_to_tensor(state))
            current_q_value = q_values[0, action]
            target_q_value = tf.convert_to_tensor(target_q_value, dtype=tf.float32)

            current_q_value = tf.expand_dims(current_q_value, axis=0)
            target_q_value = tf.expand_dims(target_q_value, axis=0)
            
            # Calcular la pérdida
            loss = loss_fn(current_q_value, target_q_value)

        # Actualizar la red principal
        grads = tape.gradient(loss, dqn.trainable_variables)
        optimizer.apply_gradients(zip(grads, dqn.trainable_variables))

        # Mover al siguiente estado
        state = next_state

    # Actualizar la red objetivo cada 'target_update_frequency' episodios
    if episode % target_update_frequency == 0:
        target_dqn.set_weights(dqn.get_weights())

    # Imprimir la pérdida cada 1000 episodios
    # if episode % 1000 == 0:
    print(f"Episodio: {episode}, Pérdida: {loss.numpy()}")

# Mostrar la matriz de recompensas para referencia
print("\nMatriz de Recompensas:")
print(rewards)

c:\Users\usuario\anaconda3\envs\env_101\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episodio: 0, Pérdida: 9937.7158203125
Episodio: 1, Pérdida: 9871.63671875
Episodio: 2, Pérdida: 9731.8349609375
Episodio: 3, Pérdida: 9740.2734375
Episodio: 4, Pérdida: 9599.759765625
Episodio: 5, Pérdida: 9391.3193359375
Episodio: 6, Pérdida: 8986.7265625
Episodio: 7, Pérdida: 9662.7626953125
Episodio: 8, Pérdida: 9391.9912109375
Episodio: 9, Pérdida: 8703.8916015625

Matriz de Recompensas:
[[ -0.1  -0.1  -0.1  -0.1]
 [ -0.1  -1.   -0.1   1. ]
 [ -0.1  -0.1   1.   -0.1]
 [ -1.   -0.1  -0.1 100. ]]


# Experience replay

In [26]:
from collections import deque

# Parámetros
memory_size = 10000
batch_size = 64

In [71]:
# import tensorflow as tf
# # Configuración de paralelismo
# # tf.config.threading.set_inter_op_parallelism_threads(4)
# # tf.config.threading.set_intra_op_parallelism_threads(4)


# # Parámetros
# grid_size = 4
# learning_rate = 0.001
# discount_factor = 0.99
# epsilon = 0.1
# episodes = 100
# target_update_frequency = episodes // 10
# memory_size = 10000
# batch_size = 64

# # Definir la red neuronal para aproximar la función Q
# class DQN(tf.keras.Model):
#     def __init__(self):
#         super(DQN, self).__init__()
#         self.fc1 = tf.keras.layers.Dense(128, activation='relu', input_shape=(grid_size * grid_size,))
#         self.fc2 = tf.keras.layers.Dense(128, activation='relu')
#         self.fc3 = tf.keras.layers.Dense(4)  # 4 acciones posibles: arriba, abajo, izquierda, derecha

#     def call(self, x):
#         x = self.fc1(x)
#         x = self.fc2(x)
#         return self.fc3(x)

# # Inicializar la red DQN principal y la red objetivo
# # Inicializar la red DQN principal y la red objetivo
# dqn = DQN()
# target_dqn = DQN()
# target_dqn.set_weights(dqn.get_weights())  # Inicializar la red objetivo con los pesos de la red principal
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# loss_fn = tf.keras.losses.MeanSquaredError()

# # Memoria de repetición (Experience Replay Memory)
# replay_memory = deque(maxlen=memory_size)

# # Función para convertir el estado (posición en la cuadrícula) en un tensor de entrada para la red
# def state_to_tensor(state):
#     tensor = np.zeros(grid_size * grid_size)
#     tensor[state[0] * grid_size + state[1]] = 1.0
#     return np.expand_dims(tensor, axis=0).astype(np.float32)

# # Función para elegir la acción (con epsilon-greedy)
# @tf.function
# def choose_action(state):
#     if random.uniform(0, 1) < epsilon:
#         return tf.convert_to_tensor(random.randint(0, 3), dtype=tf.int64)  # Acción aleatoria
#     else:
#         q_values = dqn(state_to_tensor(state))
#         return tf.argmax(q_values[0])

# # Función para obtener la próxima posición dada una acción
# def take_action(state, action):
#     if action == 0:  # up
#         return (max(state[0] - 1, 0), state[1])
#     elif action == 1:  # down
#         return (min(state[0] + 1, grid_size - 1), state[1])
#     elif action == 2:  # left
#         return (state[0], max(state[1] - 1, 0))
#     elif action == 3:  # right
#         return (state[0], min(state[1] + 1, grid_size - 1))

# # Función de entrenamiento con Double DQN y Experience Replay
# @tf.function
# def train_step():
#     minibatch = random.sample(replay_memory, batch_size)

#     states = np.array([state_to_tensor(exp[0]) for exp in minibatch])
#     actions = np.array([exp[1] for exp in minibatch])
#     rewards = np.array([exp[2] for exp in minibatch])
#     next_states = np.array([state_to_tensor(exp[3]) for exp in minibatch])
#     dones = np.array([exp[4] for exp in minibatch])

#     states = tf.convert_to_tensor(states, dtype=tf.float32)
#     actions = tf.convert_to_tensor(actions, dtype=tf.int64)
#     rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
#     next_states = tf.convert_to_tensor(next_states, dtype=tf.float32)
#     dones = tf.convert_to_tensor(dones, dtype=tf.float32)

#     # Asegurar que las dimensiones son correctas
#     states = tf.squeeze(states, axis=1)  # Convierte de (64, 1, 16) a (64, 16)
#     next_states = tf.squeeze(next_states, axis=1)  # Convierte de (64, 1, 16) a (64, 16)

#     with tf.GradientTape() as tape:
#         # Obtener los valores Q actuales
#         q_values = dqn(states)
#         q_values = tf.reduce_sum(q_values * tf.one_hot(actions, 4), axis=1)

#         # Calcular los valores Q objetivos
#         next_q_values = target_dqn(next_states)
#         best_next_q_values = tf.reduce_max(next_q_values, axis=1)
#         target_q_values = rewards + discount_factor * best_next_q_values * (1.0 - dones)

#         # Calcular la pérdida
#         loss = loss_fn(target_q_values, q_values)

#     # Actualizar la red principal
#     grads = tape.gradient(loss, dqn.trainable_variables)
#     optimizer.apply_gradients(zip(grads, dqn.trainable_variables))

#     return loss

# episodes = 100
# target_update_frequency = episodes // 5
# # Entrenamiento con Double DQN
# for episode in range(episodes):
    
#     state = (random.randint(0, 0), random.randint(0, 0))

#     # max_steps = batch_size * 100
#     while state != (3, 3):  # Suponiendo que la meta es (3, 3)
#         action = choose_action(state)
#         next_state = take_action(state, action)

#         # Obtener la recompensa correspondiente
#         reward = rewards[next_state[0], next_state[1]]
#         done = (next_state == (3, 3))

#         # Almacenar la experiencia en la memoria de repetición
#         replay_memory.append((state, action.numpy(), reward, next_state, done))

#         # Actualizar el estado actual
#         state = next_state

#         # Solo entrenar si hay suficientes experiencias en la memoria
#         if len(replay_memory) >= batch_size:
#             loss = train_step()
#             #print("training")

#         # max_steps -= 1

#     # Actualizar la red objetivo cada 'target_update_frequency' episodios
#     if episode % target_update_frequency == 0:
#         target_dqn.set_weights(dqn.get_weights())

#     # Imprimir la pérdida cada 1000 episodios
#     # if episode % 1000 == 0:
#     print(f"Episodio: {episode}, Pérdida: {loss.numpy()}")

# # Mostrar la matriz de recompensas para referencia
# print("\nMatriz de Recompensas:")
# print(rewards)


In [91]:
import tensorflow as tf
# Configuración de paralelismo
# tf.config.threading.set_inter_op_parallelism_threads(4)
# tf.config.threading.set_intra_op_parallelism_threads(4)


# Parámetros
grid_size = 4
learning_rate = 0.001
discount_factor = 0.99
epsilon = 0.1
episodes = 100
target_update_frequency = episodes // 10
memory_size = 1000
batch_size = 16

# Definir la red neuronal para aproximar la función Q
class DQN(tf.keras.Model):
    def __init__(self):
        super(DQN, self).__init__()
        self.fc1 = tf.keras.layers.Dense(16, activation='relu', input_shape=(grid_size * grid_size,))
        self.fc2 = tf.keras.layers.Dense(8, activation='relu')
        self.fc3 = tf.keras.layers.Dense(4)  # 4 acciones posibles: arriba, abajo, izquierda, derecha

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return self.fc3(x)

# Inicializar la red DQN principal y la red objetivo
dqn = DQN()
target_dqn = DQN()
target_dqn.set_weights(dqn.get_weights())  # Inicializar la red objetivo con los pesos de la red principal
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.MeanSquaredError()

# Memoria de repetición (Experience Replay Memory)
replay_memory = deque(maxlen=memory_size)

# Función para convertir el estado (posición en la cuadrícula) en un tensor de entrada para la red
def state_to_tensor(state):
    tensor = np.zeros(grid_size * grid_size)
    tensor[state[0] * grid_size + state[1]] = 1.0
    return tf.convert_to_tensor(np.expand_dims(tensor, axis=0).astype(np.float32))

# Función para elegir la acción (con epsilon-greedy)
@tf.function
def choose_action(state):
    state_tensor = state_to_tensor(state)
    if tf.random.uniform(()) < epsilon:
        return tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int64)  # Acción aleatoria
    else:
        q_values = dqn(state_tensor)
        return tf.argmax(q_values[0])

# Función para obtener la próxima posición dada una acción
def take_action(state, action):
    if action == 0:  # up
        return (max(state[0] - 1, 0), state[1])
    elif action == 1:  # down
        return (min(state[0] + 1, grid_size - 1), state[1])
    elif action == 2:  # left
        return (state[0], max(state[1] - 1, 0))
    elif action == 3:  # right
        return (state[0], min(state[1] + 1, grid_size - 1))

# Función de entrenamiento con Double DQN y Experience Replay
# @tf.function
def train_step():
    # print("train_step")
    minibatch = random.sample(replay_memory, batch_size)

    states = tf.convert_to_tensor([state_to_tensor(exp[0]) for exp in minibatch], dtype=tf.float32)
    actions = tf.convert_to_tensor([exp[1] for exp in minibatch], dtype=tf.int64)
    rewards = tf.convert_to_tensor([exp[2] for exp in minibatch], dtype=tf.float32)
    next_states = tf.convert_to_tensor([state_to_tensor(exp[3]) for exp in minibatch], dtype=tf.float32)
    dones = tf.convert_to_tensor([exp[4] for exp in minibatch], dtype=tf.float32)

    states = tf.squeeze(states, axis=1)  # Convierte de (64, 1, 16) a (64, 16)
    next_states = tf.squeeze(next_states, axis=1)  # Convierte de (64, 1, 16) a (64, 16)

    # print("GRADIENT")
    with tf.GradientTape() as tape:
        # Obtener los valores Q actuales
        q_values = dqn(states)
        q_values = tf.reduce_sum(q_values * tf.one_hot(actions, 4), axis=1)

        # Calcular los valores Q objetivos
        next_q_values = target_dqn(next_states)
        best_next_q_values = tf.reduce_max(next_q_values, axis=1)

        # print(minibatch)
        target_q_values = rewards + discount_factor * best_next_q_values * (1.0 - dones)

        # Calcular la pérdida
        loss = loss_fn(target_q_values, q_values)

    # Actualizar la red principal
    grads = tape.gradient(loss, dqn.trainable_variables)
    optimizer.apply_gradients(zip(grads, dqn.trainable_variables))

    return loss

# Entrenamiento con Double DQN
for episode in range(episodes):
    
    # state = (random.randint(0, grid_size - 1), random.randint(0, grid_size - 1))
    state = (0, 0)

    max_steps = 1000
    while state != (3, 3) and max_steps > 0:  # Suponiendo que la meta es (3, 3)
        action = choose_action(state)
        next_state = take_action(state, action)

        # Obtener la recompensa correspondiente
        reward = rewards[next_state]
        done = (next_state == (3, 3))

        # print([state, action.numpy(), next_state, reward, done])

        # Almacenar la experiencia en la memoria de repetición
        replay_memory.append((state, action.numpy(), reward, next_state, done))

        # Actualizar el estado actual
        state = next_state

        # Solo entrenar si hay suficientes experiencias en la memoria
        if len(replay_memory) >= batch_size:
            loss = train_step()
            # print("training")
        
        max_steps -= 1
    # print(state)

    # Actualizar la red objetivo cada 'target_update_frequency' episodios
    if len(replay_memory) >= batch_size and episode % target_update_frequency == 0:
        target_dqn.set_weights(dqn.get_weights())

    # Imprimir la pérdida cada 1000 episodios
    # if episode % 1000 == 0:
    print(f"Episodio: {episode}, Pérdida: {loss.numpy()}")

Episodio: 0, Pérdida: 1.4323081813927274e-06
Episodio: 1, Pérdida: 0.0005665533826686442
Episodio: 2, Pérdida: 0.022762442007660866
Episodio: 3, Pérdida: 0.0020240885205566883
Episodio: 4, Pérdida: 0.0024692073930054903
Episodio: 5, Pérdida: 0.008173024281859398
Episodio: 6, Pérdida: 0.0033509517088532448
Episodio: 7, Pérdida: 0.006048254203051329
Episodio: 8, Pérdida: 0.0034438101574778557
Episodio: 9, Pérdida: 0.05461530759930611
Episodio: 10, Pérdida: 0.02130291797220707
Episodio: 11, Pérdida: 1.347800850868225
Episodio: 12, Pérdida: 0.3518032431602478
Episodio: 13, Pérdida: 0.12438702583312988
Episodio: 14, Pérdida: 0.04423190653324127
Episodio: 15, Pérdida: 0.1933916062116623
Episodio: 16, Pérdida: 439.7012939453125
Episodio: 17, Pérdida: 1026.909912109375
Episodio: 18, Pérdida: 0.7979012131690979
Episodio: 19, Pérdida: 0.10414981842041016
Episodio: 20, Pérdida: 410.15643310546875
Episodio: 21, Pérdida: 3.1724562644958496
Episodio: 22, Pérdida: 7.418040752410889
Episodio: 23, Pérd

In [84]:
rewards

array([[ -0.1,  -0.1,  -0.1,  -0.1],
       [ -0.1,  -1. ,  -0.1,   1. ],
       [ -0.1,  -0.1,   1. ,  -0.1],
       [ -1. ,  -0.1,  -0.1, 100. ]])

In [92]:
loss.numpy()

58.68781

In [93]:
for i in range(4):
    for j in range(4):
        policy_direction = tf.argmax(dqn(state_to_tensor((i, j))), axis=1).numpy()[0]
        # print(policy_direction)
        dict_policy = {
            0: "arriba",
            1: "abajo",
            2: "izquierda",
            3: "derecha",
        }
        print(f"Q({i},{j}): {dict_policy[policy_direction]}")

Q(0,0): abajo
Q(0,1): abajo
Q(0,2): abajo
Q(0,3): abajo
Q(1,0): abajo
Q(1,1): abajo
Q(1,2): abajo
Q(1,3): abajo
Q(2,0): derecha
Q(2,1): abajo
Q(2,2): abajo
Q(2,3): abajo
Q(3,0): derecha
Q(3,1): derecha
Q(3,2): derecha
Q(3,3): abajo


# END